In [2]:
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pandas as pd
import pathway as pw

In [5]:
df = pd.read_csv('dataset.csv')
df.head()

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


In [6]:
# i'll start by making a single time-related column
df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
    format='%d-%m-%Y %H:%M:%S'
)

# i'll sort the data by timestamp so the real-time data can be processed chronologically
df = df.sort_values('Timestamp').reset_index(drop=True)

# for model 1, i'll save only the relevant columns Occupancy and Capacity into a separate df
df[['SystemCodeNumber','Timestamp','Occupancy','Capacity']].to_csv('parking_stream.csv', index=False)

In [7]:
# defining schema to structure the data i'm putting into pathway
class ParkingSchema(pw.Schema):
  SystemCodeNumber: str
  Timestamp: str
  Occupancy: int
  Capacity: int

# load the data as a stream
data = pw.demo.replay_csv(
    'parking_stream.csv',
    schema=ParkingSchema,
    input_rate=1000
)

In [8]:
# converting Timestamp values into proper datetime format
format = '%Y-%m-%d %H:%M:%S'
data_with_time = data.with_columns(
    t=data.Timestamp.dt.strptime(format),
    day=data.Timestamp.dt.strptime(format).dt.strftime('%Y-%m-%dT00:00:00')
)

In [10]:
alpha = 0.5 # can be adjusted

model1_output = data_with_time.select(
    pw.this.SystemCodeNumber,
    pw.this.Timestamp,
    pw.this.Occupancy,
    pw.this.Capacity
).with_columns(
    price=10 + alpha * (pw.this.Occupancy / pw.this.Capacity)
)

pd_df = pw.debug.table_to_pandas(model1_output)
print(pd_df.head(5))

                            SystemCodeNumber            Timestamp  Occupancy  \
^5YWMMXYGPB26GV9S12CW38PNFW      BHMNCPHST01  2016-10-04 07:59:00        237   
^TAB8G9DFWW37PP54SXR2P79NHG      BHMNCPNST01  2016-10-04 07:59:00        249   
^YJGA9S7ASMCYDDHJ5AV4JGBFQC      BHMMBMMBX01  2016-10-04 07:59:00        264   
^MBW4V691VG4BRK96V8QCWJY9QM      BHMBCCMKT01  2016-10-04 07:59:00         61   
^6HNVX3XJR6TP92XKT2NP2XEEQR         Shopping  2016-10-04 07:59:00        614   

                             Capacity      price  
^5YWMMXYGPB26GV9S12CW38PNFW      1200  10.098750  
^TAB8G9DFWW37PP54SXR2P79NHG       485  10.256701  
^YJGA9S7ASMCYDDHJ5AV4JGBFQC       687  10.192140  
^MBW4V691VG4BRK96V8QCWJY9QM       577  10.052860  
^6HNVX3XJR6TP92XKT2NP2XEEQR      1920  10.159896  


In [11]:
# NOTE: model1 intentionally uses only some features as per the baseline requirements
# model2 will incorporate other features

In [12]:
# it worked!! now using bokeh to visualise

from bokeh.plotting import figure, show, output_notebook

output_notebook() # hoping bokeh will output inline in colab

def plot_prices(df):
  fig = figure(
      height=400,
      width=800,
      title='Model 1: Real-Time Pricing',
      x_axis_type='datetime',
      x_axis_label='Time',
      y_axis_label='Price (dollars)'
  )
  fig.line(df['Timestamp'], df['price'], line_width=2, color='navy')
  fig.circle(df['Timestamp'], df['price'], size=6, color='red')
  return fig

# converting pathway table to pandas so i can output a static plot
static_data = pw.debug.table_to_pandas(model1_output)

plot_df = pd.DataFrame({
    'Timestamp': pd.to_datetime(static_data['Timestamp']),
    'price': static_data['price'].astype(float)
})

show(plot_prices(plot_df))

In [13]:
results = pw.debug.table_to_pandas(model1_output)

display(results[['SystemCodeNumber','Timestamp', 'Occupancy', 'Capacity', 'price']].head(10))

,SystemCodeNumber,Timestamp,Occupancy,Capacity,price
^5YWMMXYGPB26GV9S12CW38PNFW,BHMNCPHST01,2016-10-04 07:59:00,237,1200,10.098750
^MBW4V691VG4BRK96V8QCWJY9QM,BHMBCCMKT01,2016-10-04 07:59:00,61,577,10.052860
^TAB8G9DFWW37PP54SXR2P79NHG,BHMNCPNST01,2016-10-04 07:59:00,249,485,10.256701
^YJGA9S7ASMCYDDHJ5AV4JGBFQC,BHMMBMMBX01,2016-10-04 07:59:00,264,687,10.192140
^66TAC1KYJMKMM0W9FF9TAD9GQ0,BHMEURBRD01,2016-10-04 07:59:00,117,470,10.124468
^6HNVX3XJR6TP92XKT2NP2XEEQR,Shopping,2016-10-04 07:59:00,614,1920,10.159896
^NT40QE3DRNKRQG0XWTKGGR9YS8,Others-CCCPS8,2016-10-04 07:59:00,445,1322,10.168306
^ER9FDCM2TZ0EHMYB972J9MNPVR,Broad Street,2016-10-04 07:59:00,178,690,10.128986
^Y6MKAJVPGFPP7QQY239EMV3QVM,Others-CCCPS105a,2016-10-04 07:59:00,709,2009,10.176456
^V7VAYFB6FVZACH0QXH8JK2JPMM,BHMBCCTHL01,2016-10-04 07:59:00,120,387,10.155039


In [16]:
# i didn't like the graph plot, that's why i'm trying this

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import gridplot

plot_data = pd.DataFrame({
    'Timestamp': pd.to_datetime(results['Timestamp']),
    'SystemCodeNumber': results['SystemCodeNumber'],
    'price': results['price'].astype(float)
})

# i want to group by the parking lot (systemcodenumber)
grouped = plot_data.groupby('SystemCodeNumber')

# making separate plots for each parking lot
plots = []
for name, group in grouped:
  p = figure(width=300, height=200, title=name, x_axis_type='datetime')
  p.line(group['Timestamp'], group['price'], line_width=2)
  plots.append(p)

show(gridplot(plots, ncols=3))

In [17]:
# that graph plot isn't great either... let's try smaller scale ones

# 1. daily price trends (per parking lot)

from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import gridplot

# adding an hour column
results['Hour'] = pd.to_datetime(results['Timestamp']).dt.hour + pd.to_datetime(results['Timestamp']).dt.hour

# selecting only one day's data
sample_date = '2016-10-04'
daily_data = results[pd.to_datetime(results['Timestamp']).dt.date == pd.to_datetime(sample_date).date()]

# one plot per parking plot
plots = []
for lot_id, group in daily_data.groupby('SystemCodeNumber'):
  # calculating hourly avg
  hourly_avg = group.groupby('Hour')['price'].mean().reset_index()

  p = figure(width=400, height=250,
             title=f'{lot_id} - {sample_date}',
             x_axis_label='Hour of Day',
             y_axis_label='Price (dollars)',
             tools='pan,wheel_zoom,reset')

  hover = HoverTool(tooltips=[('Time', '@Hour{0.00}'), ('Price', '@price{$0.00}')])
  p.add_tools(hover)

  p.line(hourly_avg['Hour'], hourly_avg['price'],
         line_width=3, color='navy', legend_label='Avg Price')
  p.circle(hourly_avg['Hour'], hourly_avg['price'],
           size=8, color='red', alpha=0.5)

  p.xaxis.ticker = list(range(8,17)) # 8 am to 5 pm
  plots.append(p)

show(gridplot(plots, ncols=4))

In [18]:
# 2. competitor price comparison boxplots

import numpy as np

from bokeh.models import Whisker
from bokeh.transform import factor_cmap

results['Timestamp'] = pd.to_datetime(results['Timestamp'])
sample_date = '2016-10-04'  # chose a random date to compare competitor prices

# chose the most probable busiest time, 12 to 2 pm
time_mask = (
    (results['Timestamp'].dt.time >= pd.to_datetime('12:00:00').time()) &
    (results['Timestamp'].dt.time <= pd.to_datetime('14:00:00').time()) &
    (results['Timestamp'].dt.date == pd.to_datetime(sample_date).date())
)
comparison_data = results[time_mask].copy()

# calculate boxplot statistics
def boxplot_stats(df):
    stats = df.groupby('SystemCodeNumber')['price'].agg([
        ('q1', lambda x: np.percentile(x, 25)),
        ('q2', lambda x: np.percentile(x, 50)),
        ('q3', lambda x: np.percentile(x, 75)),
        ('upper', lambda x: np.percentile(x, 75) + 1.5*(np.percentile(x, 75)-np.percentile(x, 25))),
        ('lower', lambda x: np.percentile(x, 25) - 1.5*(np.percentile(x, 75)-np.percentile(x, 25))),
        ('mean', 'mean')
    ]).reset_index()
    return stats

stats_df = boxplot_stats(comparison_data)

# create plot
p = figure(
    width=800, height=500,
    title=f"Competitor Pricing Distribution (12PM-2PM, {sample_date})",
    x_range=list(stats_df['SystemCodeNumber'].unique()),
    x_axis_label='Parking Lot',
    y_axis_label='Price ($)',
    tools="pan,wheel_zoom,reset,save"
)

# giving each lot different colours
colors = factor_cmap(
    'SystemCodeNumber',
    palette=['#3288bd', '#99d594', '#e6f598', '#fee08b', '#fc8d59', '#d53e4f'],
    factors=list(stats_df['SystemCodeNumber'].unique())
)

# IQR box
p.vbar(
    x='SystemCodeNumber', width=0.7, bottom='q1', top='q3',
    fill_color=colors, line_color='black',
    source=ColumnDataSource(stats_df)
)

# median line
p.segment(
    x0='SystemCodeNumber', y0='q2', x1='SystemCodeNumber', y1='q2',
    line_color='black', line_width=3,
    source=ColumnDataSource(stats_df)
)

# whiskers
p.add_layout(Whisker(
    source=ColumnDataSource(stats_df),
    base='SystemCodeNumber', upper='upper', lower='lower',
    line_color='black', line_width=1.5
))

# mean marker
p.circle(
    x='SystemCodeNumber', y='mean',
    size=12, color='black', fill_alpha=0.6,
    source=ColumnDataSource(stats_df)
)

# didn't look very easy to read so added this
p.xaxis.major_label_orientation = np.pi/4  # to rotate labels 45 degrees
p.xgrid.grid_line_color = None
p.y_range.start = max(5, stats_df['lower'].min() - 1)
p.y_range.end = min(20, stats_df['upper'].max() + 1)

hover = HoverTool(tooltips=[
    ("Lot", "@SystemCodeNumber"),
    ("Median", "@q2{$0.00}"),
    ("Mean", "@mean{$0.00}"),
    ("Q1-Q3", "@q1{$0.00}-@q3{$0.00}")
])
p.add_tools(hover)

show(p)

In [19]:
# 3. summary plot of hourly avgs across all lots

from bokeh.transform import linear_cmap
from bokeh.palettes import Viridis256

hourly_avg = results.groupby(
    pd.to_datetime(results['Timestamp']).dt.hour)['price'].mean().reset_index()

# creating the summary plot
p = figure(
    width=800,
    height=400,
    title="Average Hourly Price (All Lots)",
    x_axis_label='Hour of Day',
    y_axis_label='Avg Price ($)',
    tools="hover,save"
)

p.vbar(
    x='Timestamp',
    top='price',
    width=0.8,
    source=hourly_avg,
    fill_color=linear_cmap('price', Viridis256, hourly_avg['price'].min(), hourly_avg['price'].max())
)

hover = p.select_one(HoverTool)
hover.tooltips = [
    ("Hour", "@Timestamp{00}:00"),
    ("Avg Price", "@price{$0.00}")
]

show(p)